In [2]:
import joblib
import numpy as np
import pandas as pd
import pickle

In [13]:
usar_google_colab=0
if usar_google_colab==1:
# se for usar google colab
    from google.colab import files
    import io
    uploaded = files.upload()
    data = io.BytesIO(uploaded['Input_Capacidade_de_Sulfeto.csv']) #Colocar somente o nome do arquivo e nao o caminho
    df = pd.read_csv("Input_Capacidade_de_Sulfeto.csv",sep=",",decimal=".")
    uploaded = files.upload()
    min_max_scaler_x = io.BytesIO(uploaded['min_max_scaler_x'])
    min_max_scaler_x = joblib.load(min_max_scaler_x)
    uploaded = files.upload()
    min_max_scaler_y = io.BytesIO(uploaded['min_max_scaler_y'])
    min_max_scaler_y = joblib.load(min_max_scaler_y)
    uploaded = files.upload()
    network = io.BytesIO(uploaded['network_tanh-1a1'])
else:  
    df = pd.read_csv('Input_Capacidade_de_Sulfeto.csv',sep=",",decimal=".")
    min_max_scaler_x = joblib.load("min_max_scaler_x.csv")
    min_max_scaler_y = joblib.load("min_max_scaler_y.csv")

In [14]:
x_simulacao = pd.DataFrame(df.iloc[:,:7])
x_simulacao_norm = min_max_scaler_x.fit_transform(x_simulacao)

In [15]:
#Carrega o modelo
filename = 'network_tanh-1a1.csv'
NN = pickle.load(open(filename, 'rb'))

In [16]:
#executa a predição do modelo
y_predicted_norm = NN.predict(x_simulacao_norm)
y_predicted_norm = pd.DataFrame(y_predicted_norm)

#Desnormaliza o resultado do modelo
y_predict_unnorm = min_max_scaler_y.inverse_transform(y_predicted_norm)

In [22]:
#Concatena os dados de entrada com o previsto pelo modelo
y_predict_unnorm = pd.DataFrame(y_predict_unnorm, columns = ['ERRO (%)'])
x_ypredicted_unnorm = pd.concat([x_simulacao, y_predict_unnorm], axis=1) #np.concatenate((x_simulacao, y_predict_unnorm), axis=1)
x_ypredicted_unnorm = pd.DataFrame(x_ypredicted_unnorm)

print('--------')
print('Output da Simulacao')
print(x_ypredicted_unnorm.to_string(formatters={'Al2O3':'{:,.3f}'.format,'SiO2':'{:,.3f}'.format,'MgO':'{:,.3f}'.format,'Cs':'{:,.3f}'.format,'Cs - Python':'{:,.3f}'.format,'ERRO (%)':'{:,.3f}'.format}))

--------
Output da Simulacao
        CaO   SiO2   MgO  Al2O3  Temperatura     Cs Cs - Python ERRO (%)
0    40.000 40.000 1.042 18.958         1773 -4.104      -4.108    0.173
1    40.000 40.000 0.208 19.792         1673 -4.552      -4.561    0.149
2    40.000 16.667 5.000 38.333         1723 -3.765      -3.748    0.334
3    40.000 40.000 3.542 16.458         1673 -4.486      -4.479    0.067
4    40.000 40.000 3.958 16.042         1773 -4.037      -4.037    0.241
5    53.750 26.250 0.000 20.000         1773 -3.266      -3.265    0.084
6    40.000 35.000 5.000 20.000         1673 -4.369      -4.365    0.132
7    47.500 32.500 0.000 20.000         1673 -4.169      -4.158    0.181
8    50.000 30.000 0.000 20.000         1773 -3.515      -3.517    0.078
9    59.250 20.750 0.000 20.000         1723 -3.173      -3.179    0.266
10   40.000 40.000 4.375 15.625         1673 -4.470      -4.459    0.162
11   46.875 33.125 0.000 20.000         1673 -4.203      -4.192    0.179
12   40.000 16.667 5.0

In [23]:
if usar_google_colab==1:
  x_ypredicted_unnorm.to_excel (r'Output_Simulacao_export_dataframe.xlsx', index = None, header=None)
  files.download('Output_Simulacao_export_dataframe.xlsx')
else:
  export_csv = x_ypredicted_unnorm.to_csv ('Output_Simulacao_export_dataframe.csv', index = None, header=True)
  export_excel = x_ypredicted_unnorm.to_excel ('Output_Simulacao_export_dataframe.xlsx', index = None, header=None)